In [4]:
!pip install -q -U langchain langchain-community langchain-google-genai \
    chromadb yfinance pandas python-dotenv tavily-python

  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import getpass
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter your Gemini API Key: ")

if not os.getenv("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"]=getpass.getpass("Enter your Tavily API Key: ")

print("\n Keys stored in session memory.")

Enter your Gemini API Key:  ········
Enter your Tavily API Key:  ········



✅ Keys stored in session memory.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

def extract_ticker(query):
    prompt=ChatPromptTemplate.from_template(
        "You are a financial entity extractor. From the user query: '{query}', "
        "identify the primary stock ticker symbol. "
        "Return ONLY a JSON object in this format: {{\"ticker\": \"SYMBOL\"}}. "
        "If the company name is mentioned instead of the ticker, convert it to the ticker (e.g., 'Apple' -> 'AAPL'). "
        "If no company or ticker is found, return {{\"ticker\": null}}."
    )
    chain=prompt | llm | JsonOutputParser()
    result=chain.invoke({"query": query})
    return result.get("ticker")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_text_splitters import RecursiveCharacterTextSplitter

def fetch_market_context(ticker):
    search=TavilySearchResults(k=5)
    query=f"Recent financial news and price movement analysis for {ticker} stock"
    search_results=search.run(query)
    
 
    raw_text="\n\n".join([res['content'] for res in search_results])
  
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks=text_splitter.split_text(raw_text)
    
    return chunks



In [10]:
!pip install -qU langchain-chroma


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install -qU langchain-huggingface


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_chroma import Chroma 
from langchain_huggingface import HuggingFaceEmbeddings

def create_local_knowledge_base(chunks):
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(model_name=model_name)
    vectorstore=Chroma.from_texts(
        texts=chunks,
        embedding=embeddings,
        collection_name="market_news_temp"
    )
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
import yfinance as yf
import pandas as pd

def get_stock_stats(ticker):
    stock=yf.Ticker(ticker)
    
    hist=stock.history(period="1mo")
    
    if hist.empty:
        return "No historical price data found for this ticker."

    current_price=hist['Close'].iloc[-1]
    start_price=hist['Close'].iloc[0]
    pct_change=((current_price-start_price)/start_price)*100
    high_price=hist['High'].max()
    low_price=hist['Low'].min()

    stats_summary=(
        f"Financial Stats for {ticker} (Last 30 Days):\n"
        f"- Current Price: ${current_price:.2f}\n"
        f"- 30-Day Change: {pct_change:.2f}%\n"
        f"- 30-Day High: ${high_price:.2f}\n"
        f"- 30-Day Low: ${low_price:.2f}\n"
    )
    
    return stats_summary



In [21]:
# Install the legacy compatibility package
!pip install -qU langchain-classic


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
!pip install -q -U langchain langchain-core


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

def run_stock_analyst(user_query):
    ticker=extract_ticker(user_query)
    if not ticker:
        return "I couldn't identify a stock ticker in your request. Please try again (e.g., 'Why did Apple move today?')."
    news_chunks=fetch_market_context(ticker)
    retriever=create_local_knowledge_base(news_chunks)
    
    stats = get_stock_stats(ticker)
    
    system_prompt = (
        "You are a Senior Financial Analyst. Answer the user's question using the provided context and stats.\n\n"
        "STATISTICS:\n{stats}\n\n"
        "LATEST NEWS CONTEXT:\n{context}\n\n"
        "INSTRUCTIONS:\n"
        "- Use the stats to confirm or contrast what the news says.\n"
        "- If the price dropped but news is positive, point out the divergence.\n"
        "- Keep the tone professional and data-driven."
    )
    
    prompt=ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])
    question_answer_chain=create_stuff_documents_chain(llm, prompt)
    rag_chain=create_retrieval_chain(retriever, question_answer_chain)
    
    response=rag_chain.invoke({"input": user_query, "stats": stats})
    
    return response["answer"]



In [ ]:
import time

def main():
    print("="*60)
    print(" AI FINANCIAL ANALYST TERMINAL ACTIVE")
    print("Enter a company name or question (e.g., 'What's happening with Tesla?')")
    print("Type 'exit' or 'q' to stop.")
    print("="*60)

    while True:
        user_input=input("\n[User]: ")
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("Terminating session... Goodbye!")
            break
        
        print(f"\n[Analyst]:  Fetching news and technical stats...")
        
        try:
            start_time=time.time()
            
            response=run_stock_analyst(user_input)
            
            end_time=time.time()
        
            print("-" * 60)
            print(response)
            print("-" * 60)
            print(f" Analysis Time: {end_time-start_time:.2f} seconds")
            
        except Exception as e:
            print(f" ERROR: {str(e)}")
            print("Check your API keys and internet connection.")
if __name__ == "__main__":
    main()

 AI FINANCIAL ANALYST TERMINAL ACTIVE
Enter a company name or question (e.g., 'What's happening with Tesla?')
Type 'exit' or 'q' to stop.



[User]:  Why did Apple stock drop?



[Analyst]:  Fetching news and technical stats...
------------------------------------------------------------
Apple (AAPL) stock has experienced a recent decline, dropping by $8.83 (3.46%) since the market last closed, and is down approximately 3% over the past five days, contributing to an 8.96% decrease over the last 30 days.

The primary factors contributing to this drop include:
*   **Near-term concerns:** Investors are weighing concerns around **cost pressures** and **Services growth**.
*   **Analyst price target adjustment:** Citi analyst Atif Malik lowered the firm’s price target on Apple to $315 from $330, although the firm maintained a "Buy" rating on the shares.
*   **Broader market weakness:** US stocks recently suffered their worst day since October, indicating a general downturn that could impact individual stocks like AAPL.

This decline occurs despite some positive news, such as a Paris court ruling allowing Apple to keep its ATT feature. While the stock has seen a rece


[User]:  exit


Terminating session... Goodbye!
